In [1]:
import os

os.getcwd()

'C:\\Users\\administrator'

# 1.파일 읽기

In [2]:
from tqdm import tqdm

In [3]:
import pandas as pd
import numpy as np

# file_path = path + 'songs_data_final_2180.csv'
data1 = pd.read_csv('merged_playlists_90000.csv')

data1.head()

,song_name,artist,release_date,album,genre,lyric,playlist_name
0,사랑이 너무 헤퍼,이강승,21.03.11,Korean Dream,인디음악,사랑이 너무 헤퍼서 그 말을 못 했네 말라붙은 입에 잘못은 아니고 스쳐진 옷깃에 우...,내 손을 꼭 쥐고 들려주는 인디 노래
1,가을밤에 든 생각,잔나비,20.11.06,잔나비 소곡집 l,록/메탈,머나먼 별빛 저 별에서도 노랠 부르는 사랑 살겠지 밤이면 오손도손 그리운 것들 모아...,내 손을 꼭 쥐고 들려주는 인디 노래
2,Something To Love,전기장판,19.09.09,Something To Love,인디음악,Something to love Something to love 모난 모서리가 아프...,내 손을 꼭 쥐고 들려주는 인디 노래
3,아침,이예린,21.01.19,먼 마음 2/2,포크/블루스,솔직하게 말할게요 자신이 없어요 잘하고 싶어도 잘 안됐어요 좋게 생각하려 해도 금세...,내 손을 꼭 쥐고 들려주는 인디 노래
4,아마도 우린,우효,15.10.07,어드벤처,일렉트로니카,아마도 우린 아마도 여기까진가 봐 아무리 아닌 척 해봐도 이렇게 우린 서로에게 아픈...,내 손을 꼭 쥐고 들려주는 인디 노래


In [4]:
len(data1)

4937142

In [5]:
data1.dropna(inplace= True)

In [7]:
# 고유한 플레이리스트 이름 추출
unique_playlists = data1['playlist_name'].unique()

# 무작위로 5만 개의 플레이리스트 이름 선택
selected_playlists = np.random.choice(unique_playlists, size= 30000, replace= False)

# 선택된 플레이리스트에 해당하는 데이터만 필터링
data = data1[data1['playlist_name'].isin(selected_playlists)]

In [3]:
# data.to_csv('data_30000.csv', index=False)

NameError: name 'data' is not defined

In [4]:
import pandas as pd
import numpy as np

data = pd.read_csv('data_30000.csv')

In [6]:
len(data)

1591283

In [7]:
data['release_date'] = data['release_date'].str.extract(r'(\d{2})').astype(str)
data['release_date']

0          13
1          21
2          17
3          04
4          18
           ..
1591278    20
1591279    18
1591280    19
1591281    17
1591282    17
Name: release_date, Length: 1591283, dtype: object

In [8]:
len(data)

1591283

In [9]:
data1.head()

NameError: name 'data1' is not defined

In [10]:
data.isnull().sum()

song_name        0
artist           0
release_date     0
album            0
genre            0
lyric            0
playlist_name    0
dtype: int64

In [11]:
len(data)

1591283

## 1.5 저장경로 재설정

In [12]:
import os

os.getcwd()

'C:\\Users\\administrator'

In [13]:
%cd C:\\Users\\administrator\\전처리데이터

os.getcwd()

C:\Users\administrator\전처리데이터


'C:\\Users\\administrator\\전처리데이터'

# 2.파라미터 설정

In [14]:
max_songs = 10
playlist_length = 25
lyrics_length = 150

# 3.토큰 식별 사전 생성 함수
  토크나이저는 어절 단위로 자동적으로 분리함. 이를 위해 별개의 딕셔너리를 형성하고, 별도의 매핑을 진행함.

In [15]:
def create_identifiers(data, column_name):
    """
    Create a dictionary that maps each unique value in the specified column of the dataframe
    to a unique identifier.

    :param data: pandas DataFrame containing the data.
    :param column_name: Name of the column for which to create identifiers.
    :return: Dictionary mapping each unique value in the column to a unique identifier.
    """
    identifiers = {value: f'{column_name}_{idx}' for idx, value in enumerate(data[column_name].unique(), start=1)}
    identifiers['<sos>'] = '<sos>'
    identifiers['<eos>'] = '<eos>'
    return identifiers

song_identifiers = create_identifiers(data, 'song_name')            # decoder_input : 노래 이름
artist_identifiers = create_identifiers(data,'artist')              # concate_input
album_identifiers = create_identifiers(data,'album')
genre_identifiers = create_identifiers(data,'genre')
release_date_identifiers = create_identifiers(data,'release_date')

In [16]:
# 스페셜 토큰을 확인
release_date_identifiers['<sos>']

'<sos>'

In [17]:
song_identifiers

{'Rainy Day': 'song_name_1',
 'When This Rain Stops': 'song_name_2',
 '밤편지': 'song_name_3',
 '우산 (Feat. 윤하)': 'song_name_4',
 '장마 (Rainy Season)': 'song_name_5',
 '비와 당신': 'song_name_6',
 '사랑이 이별이 돼 가는 모습이': 'song_name_7',
 '비가 오는 날엔': 'song_name_8',
 'Rainy day (Feat. ASH ISLAND, Skinny Brown)': 'song_name_9',
 '비오는 날 뭐해': 'song_name_10',
 'To Die For': 'song_name_11',
 'Sober': 'song_name_12',
 'Rain Drop': 'song_name_13',
 '비오는 날 듣고 싶은 노래': 'song_name_14',
 'Bambi': 'song_name_15',
 '우산 (Love Song)': 'song_name_16',
 '비가오면 비가오죠': 'song_name_17',
 'Umbrella (Feat. Ashley Alisha, prettyhappy)': 'song_name_18',
 'Paris in the Rain': 'song_name_19',
 '비가 오는 날엔 (2021)': 'song_name_20',
 '장마 (Feat. 영준 of Brown Eyed Soul)': 'song_name_21',
 'Rain': 'song_name_22',
 '비 오는 날 듣기 좋은 노래 (Feat. Colde)': 'song_name_23',
 'Kiss The Rain': 'song_name_24',
 '비도 오고 그래서 (Feat. 신용재)': 'song_name_25',
 '비': 'song_name_26',
 '나의 바람 (Wind And Wish)': 'song_name_27',
 '슬픈 다짐': 'song_name_28',
 'One More Ti

## 1) 매핑 딕셔너리 저장

In [15]:
import json

# JSON 파일로 저장
with open('song_identifiers.json', 'w', encoding='utf-8') as file:
    json.dump(song_identifiers, file, ensure_ascii=False, indent=4)

# 4.토크나이저 생성, 토큰화 진행

## 1) Bert : encoder_input(playlist_title), lyric

In [18]:
# 필요한 라이브러리 임포트
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# BERT 토크나이저(encoder_playlist, decoder_lyrics) import
from transformers import BertTokenizer

In [19]:
# BERT_playlist 토크나이저 생성
playlist_title_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# lyric_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

## 2) Tokenizer_self : Concat_ Other_input

In [20]:
song_tokenizer = Tokenizer(filters= '', lower= False)
artist_tokenizer = Tokenizer(filters= '', lower= False)
album_tokenizer = Tokenizer(filters= '', lower= False)
genre_tokenizer = Tokenizer(filters= '', lower= False)
release_date_tokenizer = Tokenizer(filters= '', lower= False)

In [21]:
# 고유 식별자를 사용하여 토크나이저 피팅
song_tokenizer.fit_on_texts(song_identifiers.values())
artist_tokenizer.fit_on_texts(artist_identifiers.values())
album_tokenizer.fit_on_texts(album_identifiers.values())
genre_tokenizer.fit_on_texts(genre_identifiers.values())
release_date_tokenizer.fit_on_texts(release_date_identifiers.values())

## 3) 토크나이저 저장

In [20]:
song_tokenizer_json = song_tokenizer.to_json()

# JSON 문자열을 파일로 저장
with open('song_tokenizer_none_audio.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(song_tokenizer_json, ensure_ascii= False))

In [22]:
len(song_tokenizer.word_index)

261227

In [23]:
song_tokenizer.word_index['<sos>']

261226

In [24]:
song_tokenizer.word_index['<eos>']

261227

In [29]:
# # 토크나이저 저장

# playlist_title_tokenizer.save_pretrained('playlist_title_tokenizer')

('playlist_title_tokenizer\\tokenizer_config.json',
 'playlist_title_tokenizer\\special_tokens_map.json',
 'playlist_title_tokenizer\\vocab.txt',
 'playlist_title_tokenizer\\added_tokens.json')

## 3) Tokenizer <- 식별자 사전 적용

In [25]:
# 노래 제목을 식별자로 변환
data['song_identifier'] = data['song_name'].map(song_identifiers)

# 데이터에 고유 식별자 적용
data['artist_identifier'] = data['artist'].map(artist_identifiers)
data['album_identifier'] = data['album'].map(album_identifiers)
data['genre_identifier'] = data['genre'].map(genre_identifiers)
data['release_date_identifier'] = data['release_date'].map(release_date_identifiers)

In [26]:
len(data['artist_identifier'].unique())

64381

In [27]:
len(data['artist'].unique())

64381

In [33]:
# max(data['artist_identifier'].unique())

# 5.데이터 GruopBy 및 Shuffle

In [28]:
len(data)

1591283

In [29]:
# 데이터셋 로드
# 플레이리스트별로 그룹화
# shuffled_grouped  = data.groupby('playlist_name')
shuffled_grouped  = data.groupby('playlist_name')

In [30]:
shuffled_grouped.head(10)

,song_name,artist,release_date,album,genre,lyric,playlist_name,song_identifier,artist_identifier,album_identifier,genre_identifier,release_date_identifier
0,Rainy Day,에일리(Ailee),13,A's Doll House,발라드,이렇게 또 비가 내리면 헤어지던 때가 생각나 붙잡던 날 버리고 나를 떠난 너 너는 ...,추천가요 트롯트,song_name_1,artist_1,album_1,genre_1,release_date_1
1,When This Rain Stops,웬디 (WENDY),21,Like Water - The 1st Mini Album,발라드,혼자인 것만 같은 날들이 참 많기도 해 다 안다고 생각했던 것들이 낯설어지기도 하고...,추천가요 트롯트,song_name_2,artist_2,album_2,genre_1,release_date_2
2,밤편지,아이유,17,밤편지,발라드,이 밤 그날의 반딧불을 당신의 창 가까이 보낼게요 음 사랑한다는 말이에요 나 우리의...,추천가요 트롯트,song_name_3,artist_3,album_3,genre_1,release_date_3
3,우산 (Feat. 윤하),에픽하이 (EPIK HIGH),04,"Pieces, Part One",랩/힙합,어느새 빗물이 내 발목에 고이고 참았던 눈물이 내 눈가에 고이고 I cry 텅빈 방...,추천가요 트롯트,song_name_4,artist_4,album_4,genre_2,release_date_4
4,장마 (Rainy Season),마마무 (Mamamoo),18,장마 (Rainy Season),R&B/Soul,It's a rainy day 헤어지기엔 아직은 좀 이른 듯해 Why Why Why...,추천가요 트롯트,song_name_5,artist_5,album_5,genre_3,release_date_5
...,...,...,...,...,...,...,...,...,...,...,...,...
1591235,돌아가자,늘 그대,20,돌아가자,뉴에이지,아직 가사가 등록되지 않았습니다.,일상™ 같은 Music♬,song_name_52475,artist_16652,album_38942,genre_7,release_date_8
1591236,인연(因緣),휘월 (徽月),20,인연(因緣),뉴에이지,아직 가사가 등록되지 않았습니다.,일상™ 같은 Music♬,song_name_52505,artist_16680,album_38967,genre_7,release_date_8
1591237,달빛에 그대 비추다(美輝月),월아 (月兒),20,달빛에 그대 비추다(美輝月),뉴에이지,아직 가사가 등록되지 않았습니다.,일상™ 같은 Music♬,song_name_52515,artist_16689,album_38975,genre_7,release_date_8
1591238,마음에 기대어,감성시인,20,마음 시,뉴에이지,아직 가사가 등록되지 않았습니다.,일상™ 같은 Music♬,song_name_38700,artist_12760,album_29384,genre_7,release_date_8


In [31]:
count = 0
for name, _ in shuffled_grouped :
    if count < 2 :
        print(name)
        
    else :
        break

    count += 1 

! 띵곡 !
!! 가사주의 !! 가사가 가슴에 콕!


# 6.Encoder, Decoder_input 전처리 변환

In [32]:
song_tokenizer.word_index['<sos>']

261226

In [33]:
song_tokenizer.word_index['<eos>']

261227

In [34]:
# 플레이리스트별 데이터 그룹화 및 시퀀스 생성
playlist_sequences = []
song_sequences = []

# # [SOS]와 [EOS] 토큰의 인덱스 할당
sos_index = song_tokenizer.word_index['<sos>']
eos_index = song_tokenizer.word_index['<eos>']

playlist_sequences = []
song_sequences = []

count = 1
for name, group in tqdm(shuffled_grouped):
    # 플레이리스트 제목 시퀀스 생성 및 패딩
    playlist_seq = playlist_title_tokenizer.encode(name, add_special_tokens=False)
    playlist_seq_padded = pad_sequences([playlist_seq], maxlen=playlist_length, padding='post', truncating='post')[0]
    
    # 결합된 노래 제목을 하나의 시퀀스로 변환
    combined_songs = ' '.join(group['song_identifier'].tolist())
    song_seq = song_tokenizer.texts_to_sequences([combined_songs])[0]
    # print(conunt, song_seq)
    
    # <sos> 토큰 추가
    song_seq_with_sos = np.concatenate([[sos_index], song_seq])
    # print(conunt, song_seq_with_sos)
    
    # 패딩 적용 및 <eos> 토큰 추가
    if len(song_seq_with_sos) >= max_songs:
        # 이미 최대 길이에 도달했으므로, <eos> 토큰 추가 없이 바로 패딩
        song_seq_padded_sos = pad_sequences([song_seq_with_sos], maxlen=(max_songs - 1), padding='post', truncating='post')[0]
        # print(conunt, song_seq_padded_sos)
        
        song_seq_padded = np.append(song_seq_padded_sos, eos_index)
        # print(conunt, song_seq_padded)
    
    else:
        # <eos> 토큰 추가 가능
        song_seq_with_eos = np.append(song_seq_with_sos, eos_index)
        # print(conunt, song_seq_with_eos)
        
        song_seq_padded = pad_sequences([song_seq_with_eos], maxlen=max_songs, padding='post', truncating='post')[0]
        # print(conunt, song_seq_padded)
        
    # NumPy 배열로 변환
    playlist_seq_np_array = np.array(playlist_seq_padded, dtype=np.int32)
    song_seq_np_array = np.array(song_seq_padded, dtype=np.int32)

    # 결과 리스트에 추가
    playlist_sequences.append(playlist_seq_np_array)
    song_sequences.append(song_seq_np_array)
    # count += 1

100%|██████████████████████████████████████████████████████████████████████████| 30000/30000 [00:09<00:00, 3143.79it/s]


In [35]:
len(song_sequences[0])

10

In [36]:
len(song_sequences[1])

10

In [49]:
# 플레이리스트_인코더 sequence
playlist_title_tokenizer.decode(playlist_sequences[1])

'!! 가사주의!! 가사가 가슴에 콕! [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [40]:
playlist_sequences[1]

array([   106,    106,   8843,  12945,  37224,    106,    106,   8843,
        49636,   8843, 119080,  10530,   9813,    106,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0])

In [45]:
# 노래 제목_디코더 sequence
convert_sequence_to_titles(song_sequences[1], song_identifiers)

['Unknown Identifier',
 'In The Air (With 석우)',
 'lovememore.',
 '현미경 (feat. 기리보이)',
 'Blurry (Feat. Dbo) (Prod. By PEEJAY)',
 '자나깨나 (Feat. 조이 of Red Velvet)',
 'gold (Feat. Dean)',
 'GANADARA (Feat. 아이유)',
 'Is You Down',
 'Unknown Identifier']

In [48]:
song_sequences[1]

array([261226, 141929,   5819,  30198,  13103,  13821,  13126,   1934,
        30638, 261227])

In [43]:
# idetifier_ 노래이름을 토크나이저로 변환 후, 다시 노래제목으로 처리하기 위한 딕셔너리

def find_key_by_value(value_to_find, dictionary):
    for key, value in dictionary.items():
        if value == value_to_find:
            return key
    return "Unknown Identifier"  # 값이 사전에 없는 경우

# 예시 사용
# identifier = 'song_name_1202'
# song_title = find_key_by_value(identifier, song_identifiers)
# print("노래 제목:", song_title)

def convert_sequence_to_titles(sequence, identifier_to_song):
    song_titles = []
    for seq in sequence:
        
        # 숫자 인덱스를 식별자 형태로 변환
        identifier = f'song_name_{seq}'
        
        # 식별자를 노래 제목으로 변환
        song_title = find_key_by_value(identifier, identifier_to_song)
        song_titles.append(song_title)
    return song_titles

In [43]:
song_sequences[2]

array([261226,  10999,  11000,  11001,  11002,  11003,  11004,  11005,
        11006, 261227])

In [44]:
playlist_sequences[0]

array([106, 100, 106,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [45]:
len(playlist_sequences[1])

25

## target_targets : Teacher Forcing 데이터 생성

In [46]:
# target_target 생성
song_target_targets = []

for seq in song_sequences:
    # 각 시퀀스에서 첫 번째 요소 (SOS 토큰)를 제거합니다.
    # 이미 EOS 토큰이 추가되어 있으므로, 별도로 추가할 필요가 없습니다.
    target_target_seq = seq[1:]
    
    # NumPy 배열로 변환
    target_target_np_array = np.array(target_target_seq, dtype=np.int32)
    target_target_np_array = np.append(target_target_np_array, 0)
    
    # target_target 리스트에 추가
    song_target_targets.append(target_target_np_array)

# 이제 target_targets 리스트는 target_target 시퀀스를 담고 있습니다.

In [47]:
song_sequences[0]

array([261226,  11931,   2162,   6936,  51948,  82571,  47283,  82572,
        64810, 261227])

In [48]:
song_target_targets[0]

array([ 11931,   2162,   6936,  51948,  82571,  47283,  82572,  64810,
       261227,      0])

In [49]:
len(song_target_targets[3])

10

# 7.Concat, Other_input 전처리

In [50]:
# 플레이리스트별 데이터 그룹화 및 시퀀스 생성
artist_sequences = []
album_sequences = []
genre_sequences = []
release_date_sequences = []

# # [SOS]와 [EOS] 토큰의 인덱스 할당
artist_sos_index = artist_tokenizer.word_index['<sos>']
artist_eos_index = artist_tokenizer.word_index['<eos>']

album_sos_index = album_tokenizer.word_index['<sos>']
album_eos_index = album_tokenizer.word_index['<eos>']

genre_sos_index = genre_tokenizer.word_index['<sos>']
genre_eos_index = genre_tokenizer.word_index['<eos>']

release_date_sos_index = release_date_tokenizer.word_index['<sos>']
release_date_eos_index = release_date_tokenizer.word_index['<eos>']

for name, group in tqdm(shuffled_grouped):
    count += 1

    # 각 플레이리스트의 아티스트, 앨범, 장르, 발매일을 하나의 문자열로 결합
    combined_artist = ' '.join(group['artist_identifier'].tolist())
    combined_album = ' '.join(group['album_identifier'].tolist())
    combined_genre = ' '.join(group['genre_identifier'].tolist())
    combined_release_date = ' '.join(group['release_date_identifier'].tolist())

    # 결합된 정보를 하나의 시퀀스로 변환
    artist_seq = np.array(artist_tokenizer.texts_to_sequences([combined_artist])[0], dtype=np.int32)
    album_seq = np.array(album_tokenizer.texts_to_sequences([combined_album])[0], dtype=np.int32)
    genre_seq = np.array(genre_tokenizer.texts_to_sequences([combined_genre])[0], dtype=np.int32)
    release_date_seq = np.array(release_date_tokenizer.texts_to_sequences([combined_release_date])[0], dtype=np.int32)

    # <sos> 토큰 추가
    artist_seq_with_sos = np.concatenate([[artist_sos_index], artist_seq])
    album_seq_with_sos = np.concatenate([[album_sos_index], album_seq])
    genre_seq_with_sos = np.concatenate([[genre_sos_index], genre_seq])
    release_date_seq_with_sos = np.concatenate([[release_date_sos_index], release_date_seq])

    # 패딩 적용 및 <eos> 토큰 추가
    for seq, eos_index in [(artist_seq_with_sos, artist_eos_index), 
                           (album_seq_with_sos, album_eos_index), 
                           (genre_seq_with_sos, genre_eos_index), 
                           (release_date_seq_with_sos, release_date_eos_index)]:
        if len(seq) >= max_songs:
            seq_padded = pad_sequences([seq], maxlen=(max_songs - 1), padding='post', truncating='post')[0]
            seq_padded = np.append(seq_padded, eos_index)
        else:
            seq_with_eos = np.append(seq, eos_index)
            seq_padded = pad_sequences([seq_with_eos], maxlen=max_songs, padding='post', truncating='post')[0]

        # 결과 리스트에 추가
        if seq is artist_seq_with_sos:
            artist_sequences.append(seq_padded)
        elif seq is album_seq_with_sos:
            album_sequences.append(seq_padded)
        elif seq is genre_seq_with_sos:
            genre_sequences.append(seq_padded)
        else:
            release_date_sequences.append(seq_padded)

100%|██████████████████████████████████████████████████████████████████████████| 30000/30000 [00:10<00:00, 2993.71it/s]


In [51]:
song_sequences[0]

array([261226,  11931,   2162,   6936,  51948,  82571,  47283,  82572,
        64810, 261227])

In [52]:
artist_sequences[4]

array([64382,   795,   915,   412,  1608,  4056,  2220,   915,  1354,
       64383])

In [53]:
release_date_sequences[2]

array([56,  2,  9,  8,  5,  6,  2,  2,  2, 57])

In [54]:
# target_target 생성
artist_targets = []

for seq in artist_sequences:
    # 각 시퀀스에서 첫 번째 요소 (SOS 토큰)를 제거합니다.
    # 이미 EOS 토큰이 추가되어 있으므로, 별도로 추가할 필요가 없습니다.
    target_target_seq = seq[1:]
    
    # NumPy 배열로 변환
    target_target_np_array = np.array(target_target_seq, dtype=np.int32)
    target_target_np_array = np.append(target_target_np_array, 0)
    
    # target_target 리스트에 추가
    artist_targets.append(target_target_np_array)

# 이제 target_targets 리스트는 target_target 시퀀스를 담고 있습니다.

In [55]:
artist_sequences[0]

array([64382,   353,   353,   353,  9783,  9783,  4769, 24091,  4769,
       64383])

In [56]:
artist_targets[0]

array([  353,   353,   353,  9783,  9783,  4769, 24091,  4769, 64383,
           0])

In [57]:
# target_target 생성
album_targets = []

for seq in album_sequences:
    # 각 시퀀스에서 첫 번째 요소 (SOS 토큰)를 제거합니다.
    # 이미 EOS 토큰이 추가되어 있으므로, 별도로 추가할 필요가 없습니다.
    target_target_seq = seq[1:]
    
    # NumPy 배열로 변환
    target_target_np_array = np.array(target_target_seq, dtype=np.int32)
    target_target_np_array = np.append(target_target_np_array, 0)
    
    # target_target 리스트에 추가
    album_targets.append(target_target_np_array)

# 이제 target_targets 리스트는 target_target 시퀀스를 담고 있습니다.

In [58]:
album_sequences[0]

array([167464,   3217,   1735,     99,  38616,  38616,  24412,  59200,
        24408, 167465])

In [59]:
album_targets[0]

array([  3217,   1735,     99,  38616,  38616,  24412,  59200,  24408,
       167465,      0])

In [60]:
# target_target 생성
genre_targets = []

for seq in genre_sequences:
    # 각 시퀀스에서 첫 번째 요소 (SOS 토큰)를 제거합니다.
    # 이미 EOS 토큰이 추가되어 있으므로, 별도로 추가할 필요가 없습니다.
    target_target_seq = seq[1:]
    
    # NumPy 배열로 변환
    target_target_np_array = np.array(target_target_seq, dtype=np.int32)
    target_target_np_array = np.append(target_target_np_array, 0)
    
    # target_target 리스트에 추가
    genre_targets.append(target_target_np_array)

# 이제 target_targets 리스트는 target_target 시퀀스를 담고 있습니다.

In [61]:
genre_sequences[0]

array([77,  3,  3,  3,  3,  3,  3,  1,  6, 78])

In [62]:
genre_targets[0]

array([ 3,  3,  3,  3,  3,  3,  1,  6, 78,  0])

In [63]:
# target_target 생성
release_date_targets = []

for seq in release_date_sequences:
    # 각 시퀀스에서 첫 번째 요소 (SOS 토큰)를 제거합니다.
    # 이미 EOS 토큰이 추가되어 있으므로, 별도로 추가할 필요가 없습니다.
    target_target_seq = seq[1:]
    
    # NumPy 배열로 변환
    target_target_np_array = np.array(target_target_seq, dtype=np.int32)
    target_target_np_array = np.append(target_target_np_array, 0)
    
    # target_target 리스트에 추가
    release_date_targets.append(target_target_np_array)

# 이제 target_targets 리스트는 target_target 시퀀스를 담고 있습니다.

In [64]:
release_date_sequences[0]

array([56, 22, 22, 22, 22, 22, 21, 10, 13, 57])

In [65]:
release_date_targets[0]

array([22, 22, 22, 22, 22, 21, 10, 13, 57,  0])

# 10.데이터저장하기

In [66]:
print(type(playlist_sequences))
print(type(song_sequences))

print(type(artist_sequences))
print(type(album_sequences))
print(type(genre_sequences))
print(type(release_date_sequences))

# print(type(lyrics_sequences))
print(type(artist_targets))
print(type(album_targets))
print(type(genre_targets))
print(type(release_date_targets))
# print(type(audio_feature_sequences))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [67]:
song_sequences[0]

array([261226,  11931,   2162,   6936,  51948,  82571,  47283,  82572,
        64810, 261227])

In [68]:
song_target_targets[0]

array([ 11931,   2162,   6936,  51948,  82571,  47283,  82572,  64810,
       261227,      0])

In [69]:
playlist_sequences_np = np.array(playlist_sequences)

song_sequences_np = np.array(song_sequences)
artist_sequences_np = np.array(artist_sequences)
album_sequences_np = np.array(album_sequences)
genre_sequences_np = np.array(genre_sequences)
release_date_sequences_np = np.array(release_date_sequences)

song_targets_np = np.array(song_target_targets)
artist_targets_np = np.array(artist_targets)
album_targets_np = np.array(album_targets)
genre_targets_np = np.array(genre_targets)
release_date_targets_np = np.array(release_date_targets)

# lyrics_sequences_np = np.array(lyrics_sequences)
# audio_feature_sequences_np = np.array(audio_feature_sequences)

In [70]:
print((playlist_sequences_np.shape))

print((song_sequences_np.shape))
print((artist_sequences_np.shape))
print((album_sequences_np.shape))
print((genre_sequences_np.shape))
print((release_date_sequences_np.shape))

print((song_targets_np.shape))
print((artist_targets_np.shape))
print((album_targets_np.shape))
print((genre_targets_np.shape))
print((release_date_targets_np.shape))

(30000, 25)
(30000, 10)
(30000, 10)
(30000, 10)
(30000, 10)
(30000, 10)
(30000, 10)
(30000, 10)
(30000, 10)
(30000, 10)
(30000, 10)


In [71]:
import numpy as np

# playlist_sequences 저장
np.save('playlist_title_input_9.npy', playlist_sequences_np)

# song_sequences 저장
np.save('song_inputs_9.npy', song_sequences_np)
# artist_sequences 저장
np.save('artist_input_9.npy', artist_sequences_np)
# album_sequences 저장
np.save('album_input_9.npy', album_sequences_np)
# genre_sequences 저장
np.save('genre_input_9.npy', genre_sequences_np)
# release_date_sequences 저장
np.save('release_date_input_9.npy', release_date_sequences_np)

# target_targets 저장
np.save('song_targets_9.npy', song_targets_np)
np.save('artist_targets_9.npy', artist_targets_np)
np.save('album_targets_9.npy', album_targets_np)
np.save('genre_targets_9.npy', genre_targets_np)
np.save('release_date_targets_9.npy', release_date_targets_np)

# lyrics_sequences 저장
# np.save('lyrics_input_9.npy', lyrics_sequences_np)
# # audio_feature_sequences 저장
# np.save('audio_features_input.npy', audio_feature_sequences_np)

# 11.하이퍼파라미터 설정

In [72]:
# 고유 식별자를 사용하여 토크나이저 피팅
print('song_tokenizer :',len(song_tokenizer.word_index))
print('artist_tokenizer :' , len(artist_tokenizer.word_index))
print('album_tokenizer :' , len(album_tokenizer.word_index))
print('genre_tokenizer :' , len(genre_tokenizer.word_index))
print('release_date_tokenizer :' , len(release_date_tokenizer.word_index))

song_tokenizer : 261227
artist_tokenizer : 64383
album_tokenizer : 167465
genre_tokenizer : 78
release_date_tokenizer : 57


In [73]:
playlist_vocab = playlist_title_tokenizer.vocab_size + 1
# lyric_vocab = int(lyrics_max_token_value) + 1

song_vocab = len(song_tokenizer.word_index) + 1
artist_vocab = len(artist_tokenizer.word_index) + 1
album_vocab = len(album_tokenizer.word_index) + 1
genre_vocab = len(genre_tokenizer.word_index) + 1
release_date_vocab = len(release_date_tokenizer.word_index) + 1

In [74]:
print('playlist_vocab :', playlist_vocab)
print("\n")
print('song_vocab :', song_vocab)
print('artist_vocab :', artist_vocab)
print('album_vocab :', album_vocab)
print('genre_vocab :', genre_vocab)
print('release_date_vocab :', release_date_vocab)

playlist_vocab : 119548


song_vocab : 261228
artist_vocab : 64384
album_vocab : 167466
genre_vocab : 79
release_date_vocab : 58
